In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import numpy as np
import datetime
from selenium.webdriver.common.action_chains import ActionChains
import xlsxwriter
from openpyxl import Workbook, load_workbook
import os
import shutil
import logging
import traceback
import re
import csv
from azure.storage.blob import BlobServiceClient
import argparse
import sys
import glob
import calendar
import json
import common_functions

In [2]:
# File paths


# date_today = datetime.date.today().strftime("%Y-%m-%d")
# output_gyg = r'output/GYG'
# archive_folder = fr'{output_gyg}/Archive'
# file_path_done =fr'output/GYG/{date_today}-DONE-GYG.csv'  
# file_path_output = fr"output/GYG - {date_today}.xlsx"
# link_file = fr'resource/GYG_links.csv'
# avg_file = fr'resource/avg-gyg.csv'
# re_run_path = fr'output/GYG/{date_today}-ReRun-GYG.csv'
# folder_path_with_txt_to_count_avg = 'Avg/GYG'
def constant_file_path():
    global output_gyg, archive_folder, link_file, logs_path, storage_account_name, storage_account_key, container_name_raw, container_name_refined,  link_file_path

    link_file_path = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Resource/LinksFuturePrice_GYG.json'
    output_gyg = r'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price'
    archive_folder = fr'{output_gyg}/Archive'
    link_file = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Resource/LinksFuturePrice_GYG.csv'
    logs_path = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/GYG/future_price'

    # Set the name of your Azure Storage account and the corresponding access key
    storage_account_name = "storagemyotas"
    storage_account_key = "vyHHUXSN761ELqivtl/U3F61lUY27jGrLIKOyAplmE0krUzwaJuFVomDXsIc51ZkFWMjtxZ8wJiN+AStbsJHjA=="

    # Set the name of the container and the desired blob name
    container_name_raw = "raw/future_price/gyg"
    container_name_refined = "refined/future_price/gyg"
    

def configure_dates_and_file_names(adutls, language):
    global date_today, extraction_date, extraction_date_save_format, local_file_path, blob_name,  file_path_logs_processed, output_file_path
    date_today = datetime.date.today().strftime("%Y-%m-%d")
    # Format the date and time as a string
    extraction_date = datetime.datetime.now().strftime('%Y-%m-%d %H:00:00')
    extraction_date_save_format = f"{extraction_date.replace(' ', '_').replace(':','-')}_{language}_{adutls}"
    # Set the path of the local file

    local_file_path = f"{output_gyg}/{extraction_date}_future_price.xlsx"
    blob_name = fr'{extraction_date_save_format}_future_price.xlsx'
    file_path_logs_processed = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/files_processed/{blob_name.split(".")[0]}'
    output_file_path = os.path.join(output_gyg, f"{extraction_date_save_format}_future_price.xlsx")


In [3]:
file_manager = common_functions.FilePathManagerFuturePrice(adults="2", city="Krakow", language="en", site="GYG")


In [4]:
file_manager.output_file_path

'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/GYG/future_price/2024-10-04_09-00-00_en_2_future_price.xlsx'

In [5]:
logger = common_functions.LoggerManagerFuturePrice(file_manager)


2024-10-04 09:52:19,052 - Statistics_logger - INFO - DUP


In [5]:
### GOT IT

# def handle_error_and_rerun(error):
# #     recipient_error = 'wojbal3@gmail.com'
#     tb = traceback.format_exc()
#     logger_err.error('An error occurred: {} on {}'.format(str(error), tb))
# #     subject = f'Error occurred - {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}'
# #     message = f'<html><body><p>Error occurred: {str(error)} on {tb}</p></body></html>'
# #     send_email(subject, message, recipient_error)

In [9]:
# def initilize_driver() -> WebDriver:
#     try:
#         logger_info.info("Initializing the Chrome driver and logging into the website")

#         # Setting up Chrome options
#         options = webdriver.ChromeOptions()
#         # options.add_experimental_option('excludeSwitches', ['enable-logging'])
#         options.add_argument('--blink-settings=imagesEnabled=false')

#         # Initialize the Chrome driver
#         # service = Service()
#         driver = webdriver.Chrome(options=options)
#         driver.maximize_window()
        
#         return driver

#     except Exception as e:
#         logger_err.error(f"An error occurred during login: {e}")
#         raise
    
# def quit_driver(driver: WebDriver) -> None:
#     driver.quit()    

In [9]:
# def change_currency(driver, currency_switcher_button, currency):
#     currency_text = currency_switcher_button.text.strip()
#     if currency in currency_text:
#         print('Current currency is ',currency)
#         return
#     else: 
#                 # hover over the currency switcher button to show the menu
#         actions = ActionChains(driver)
#         actions.move_to_element(currency_switcher_button).perform()
#         currency_switcher_button .click()
#         # wait for the EUR currency option to be clickable
#         eur_currency_option = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//li[@class='currency-modal-picker__item-parent item__currency-modal item__currency-modal--EUR']")))
#         # click on the EUR currency option to change the currency
#         eur_currency_option.click()

def save_and_erase_dataframe(df: pd.DataFrame, url_city_id, url_unique_identifier) -> pd.DataFrame:
    ## SAVE
    file_path = fr'{output_gyg}/{extraction_date_save_format}-{url_city_id}-GYG.csv' 
    df['city'] = url_city_id.split('-')[0]
    df['uid'] = url_unique_identifier
    df.to_csv(file_path, header=not os.path.exists(file_path), index=False, mode='a')
    logger_done.info(f'Sucessfully upserted {len(df)} rows to {file_path}')
    return pd.DataFrame()


def extract_options(driver, option_detials, activity_title, language, adults_amount, url, viewer) -> list:
    list_of_items = []
    #iterate through all options
    time.sleep(1)
    option_date = driver.current_url.split('date_from=')[-1].split('&')[0]
    
    for option in option_detials:
        option_title = option.find_element(By.CLASS_NAME, 'activity-option__title').text
        try:
            option_time_range = option.find_element(By.CLASS_NAME, 'activity-option__start-time-range').text
        except:
            option_time_range = 'Not listed'
        try:
            option_price_total = option.find_element(By.CLASS_NAME, 'activity-option-price-wrapper__price').text
            option_price_per_person = float(option.find_element(By.CLASS_NAME, 'activity-option-price-wrapper__price').text.replace('€', '').replace(',', '').strip()) / float(adults_amount)
        except:
            try:
                option_price_total = option.find_element(By.CLASS_NAME, 'activity-option__cart-message-text--future-availability').text
            except:
                option_price_total = option.find_element(By.CLASS_NAME, 'activity-option-cart-message-wrapper').text
            option_price_per_person = 'Not Available'
        try:
            spots_left = option.find_element(By.CSS_SELECTOR, "span[data-test-id*='activity-option-is-x-spots-left']").text
        except:
            spots_left = 'N/A'

        list_of_items.append({
            'extraction_date': extraction_date,
            'date': option_date,
            'title': activity_title,
            'tour_option': option_title,
            'time_range': option_time_range,
            'total_price': option_price_total,
            'price_per_person': option_price_per_person,
            'language': language,
            'adults': adults_amount,
            'spots_left': spots_left,
            'title_url': url,
            'viewer': viewer
            })    
        logger_done.info(f'Sucessfuly extracted | {option_title} | {option_time_range} | {option_price_total} | {option_price_per_person} |')
    return list_of_items

def process_days_not_available(option_date, activity_title, language, adults_amount, url, viewer) -> list:
    list_of_items = []
    list_of_items.append({
            'extraction_date': extraction_date,
            'date': option_date,
            'title': activity_title,
            'tour_option': "N/A",
            'time_range': "N/A",
            'total_price': "N/A",
            'price_per_person': "N/A",
            'language': language,
            'adults': adults_amount,
            'spots_left': "N/A",
            'title_url': url,
            'viewer': viewer
            }) 
    return list_of_items   

def activity_not_availabke_in_selected_language():
    logger_err.error("The language picker does not exisit and selected language is different from defauly english")



In [10]:
def check_if_current_day_done_or_partly_done(url_city_id, url_unique_identifier):

    # Check if the CSV file exisint in the latest file in folder future_price if so get the max days to complete

    # Extract the date part and the fixed part from the extraction_date_save_format
    date_part = extraction_date_save_format.split('_')[0] # --> 2024-05-27
    fixed_part = extraction_date_save_format.split('_', 2)[2] # --> en_2 (language_adults)
    # Define the file pattern with wildcards for the hour and minute components
    file_pattern = f'{output_gyg}/{date_part}_*-*-*_{fixed_part}-{url_city_id}-GYG.csv'
    file_pattern_archive = f'{archive_folder}/{date_part}_*-*-*_{fixed_part}-{url_city_id}-GYG.csv'
    logger_info.info(f'File_pattern: {file_pattern}')

    
    # Use glob to find all matching files
    matching_files = glob.glob(file_pattern)
    matching_files_archive = glob.glob(file_pattern_archive)
    logger_info.info(f'matching_files: {matching_files}')
    for file_path in matching_files:
        # Check if the file exists
        if os.path.exists(file_path):
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            df = df[df['uid'] == url_unique_identifier]
            if len(df) == 0:
                return False, None
            # Assuming 'date' is the name of the column containing the dates
            if 'date' in df.columns:
                # Convert the 'date' column to datetime format
                df['date'] = pd.to_datetime(df['date'])
                
                # Find the maximum date
                max_date = df['date'].max()
                
                # Returning True to indicate the task is already processed, and the max date
                return True, max_date.date()  # Converting datetime to date for easier comparison
        # If the file does not exist or 'date' column is not found, return False and None
    return False, None

def check_if_today_done_on_schedule(url, schedule):

    url_unique_identifier = url.split('.com/')[-1].split('-')[-1].replace('/', '')
    url_city_id = url.split('.com/')[-1].split('/')[0]
    # Extract the date part and the fixed part from the extraction_date_save_format
    date_part = extraction_date_save_format.split('_')[0] # --> 2024-05-27
    fixed_part = extraction_date_save_format.split('_', 2)[2] # --> en_2 (language_adults)
    # Define the file pattern with wildcards for the hour and minute components
    file_pattern = f'{archive_folder}/{date_part}_*-*-*_{fixed_part}-{url_city_id}-GYG.csv'
    logger_info.info(f'File_pattern: {file_pattern}')
    
    # Use glob to find all matching files
    matching_files = glob.glob(file_pattern)
    logger_info.info(f'matching_files: {matching_files}')
    ### To implement option to run more than ince a day if the shceuld is higher to run once again
    ### Rin based on scheuld which is number on which the calculte the number of runs per month
    for file_path in matching_files:
        # Check if the file exists
        if os.path.exists(file_path):
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            df = df[df['uid'] == url_unique_identifier]
            if len(df) == 0:
                return False
            else:
                return True
        # If the file does not exist or 'date' column is not found, return False and None
    return False

In [11]:
def check_for_modal_window_to_close(driver, calendar_picker):
    try:
        modal_close_button = driver.find_element(By.CSS_SELECTOR,"button[class='tfe-modal-header__close']")
        modal_close_button.click()
        time.sleep(2)
        driver.execute_script("arguments[0].scrollIntoView(true);", calendar_picker)
        calendar_picker.click()
    except:
        driver.execute_script("arguments[0].scrollIntoView(true);", calendar_picker)
        calendar_picker.click()

In [12]:
def get_future_price(driver, url, viewer, language, adults_amount,  max_days_to_complete):
    logger_info.info(f"Adults amount: {adults_amount}")
    logger_info.info(f"Language: {language}")
    logger_info.info(f"Max days to complete: {max_days_to_complete}")
    start_time_one_link = time.time()
    # df_links = pd.read_csv(link_file)
    # df_links = df_links[df_links['run'] == 1]
    # driver = initilize_driver()

    # for _, row in df_links.iterrows():
    url_id = url
    url_unique_identifier = url.split('.com/')[-1].split('-')[-1].replace('/', '')
    url_city_id = url.split('.com/')[-1].split('/')[0]
    
    
    # start_collection_date = '2024-04-29'
    start_collection_date = date_today

    date_today_obj = (datetime.datetime.strptime(start_collection_date, "%Y-%m-%d"))
    url_detials = f'?lang={language}&date_from={start_collection_date}&_pc=1,{adults_amount}'
    
    url = url + url_detials

    # Set the number of days to complete the task
    #DEFINED IN FUCNTION CALL
    # language = 'en'
    # adults_amount = '2'
    # max_days_to_complete = 60

    # Calculate the future date by adding max_days_to_complete to the current date, then extract the day number
    # and convert it to a string. This represents the day number after the specified days are added to the current date.    
    picked_max_day_number = str((date_today_obj + datetime.timedelta(days=max_days_to_complete)).day)

    # Calculate the future date by adding max_days_to_complete to the current date
    # This is used to get a complete date object for further manipulation or formatting.
    picked_max_date_obj = (date_today_obj + datetime.timedelta(days=max_days_to_complete))

    # Format the future date object as a string in the 'YYYY-MM-DD' format
    # This provides a standard date format that can be used for display or storage.
    picked_max_date = picked_max_date_obj.strftime('%Y-%m-%d')

    # Calculate the number of months to complete the task by subtracting the current month from the month
    # of the future date and adding 1. This gives the total number of months covering the period from the current date to the future date.
    # Note: This calculation assumes the task spans within a single year and is intended for short-term calculations.
    month_to_complete = picked_max_date_obj.month - datetime.datetime.now().month + 1
    current_year = datetime.datetime.now().year

    driver.get(url)
    logger_info.info(f'URL: {url} UNIQUE ID: {url_unique_identifier}')

    #Check if the language is available for that activity

    is_done, max_date_done = check_if_current_day_done_or_partly_done(url_city_id=url_city_id, url_unique_identifier=url_unique_identifier)
    if is_done:
        # Check if max_Date from dataframe is max date to collect from webpage
        logger_info.info(f'TO REMOVE @@@ {max_date_done} == {picked_max_date}')
        if max_date_done == picked_max_date_obj.date() :
            logger_done.info(f'Url was already done today with max date: {max_date_done}')
            # continue
            return f'Url was already done today with max date: {max_date_done}'
        else:
            url = url.replace(start_collection_date, max_date_done.strftime('%Y-%m-%d'))
            driver.get(url)
    #     VERIFY IF THE CURRENCY IS CORRECT
    login_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//a[@title='Profile']")))
    login_button.click()

    currency_switcher_button = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='option option-currency']")))

    change_currency(driver, currency_switcher_button, 'EUR')
    #css selector for the box where is availability
    activity_title = driver.find_element(By.CSS_SELECTOR, "h1[data-track='activity-title']").text
    css_selector_booking_tile = f"div[data-track='booking-assistant']"
    #Initiate booking tile varaible 
    booking_tile = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_booking_tile)))
    #Initiate calendar picker 
    calendar_picker = booking_tile.find_element(By.CSS_SELECTOR,"section[class='ba-dropdown ba-date-picker']")

    # css selector for the button check availability
    css_selector_check_availability = f"button[class*='js-check-availability']"
    button_check_availability = WebDriverWait(booking_tile, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_check_availability)))
    try:
        # Check if the element with the specified data-test-id exists
        element = driver.find_element(By.CSS_SELECTOR, '[data-test-id="activity-filters-primary-language-picker"]')
        if element:
            print("Element with data-test-id 'activity-filters-primary-language-picker' exists.")
    except NoSuchElementException:
        if language != "en":
            print("Element with data-test-id 'activity-filters-primary-language-picker' does not exist.")
            #Click  Check availability (once only)
            activity_not_availabke_in_selected_language()
            #Skipping the URL
            return 
        # continue

    driver.execute_script("arguments[0].click();", button_check_availability)
    try:
        WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CLASS_NAME, 'dayContainer')))
        months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
        for month in months:
            days_available = month.find_elements(By.CSS_SELECTOR, "span[data-test-id=ba-calendar-day-available]")
            for day in days_available:
                if day.text == "" or len(day.text) == 0:
                    print("Day was empty")
                    empty = True
                    continue
                # Scroll the day into view using JavaScript
                # driver.execute_script("arguments[0].scrollIntoView();", day)
                # Alternatively, you can use ActionChains to move to the element before clicking
                ActionChains(driver).move_to_element(day).perform()
                empty = False
                day.click()
                driver.execute_script("arguments[0].click();", button_check_availability)
                break
            if not empty:
                break
    except:
        print('Current date is avialable')
        pass
        # After click on the element wait for the option detials to show up
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.TAG_NAME, 'details')))
    option_detials = driver.find_elements(By.TAG_NAME, 'details')
    option_date = driver.current_url.split('date_from=')[-1].split('&')[0]
    current_date = option_date.split('-')[-1]
    print(f"Current date: {current_date}")
    list_of_items = extract_options(driver=driver, option_detials=option_detials, activity_title=activity_title, language=language, url=url_id, viewer=viewer, adults_amount=adults_amount)
    ### After extraction transform to dataframe and save it in the CSV file in case of any error in th future
    df = pd.DataFrame(list_of_items)
    # display(df)
    ## SAVE
    df = save_and_erase_dataframe(df, url_city_id, url_unique_identifier)
    try:
        calendar_picker.click()
    except:
        check_for_modal_window_to_close(driver, calendar_picker)
    #Wait until the calendar is fully loaded
    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CLASS_NAME, 'dayContainer')))
    months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
    month_done = False


    month = months[0]
    current_month_done = False
    for i in range(0, month_to_complete):
        days_to_complete = []
        if current_month_done == True:
            months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
            month = months[1]
            current_month = driver.find_elements(By.CLASS_NAME, "flatpickr-current-month")[1].text.strip()
            current_month_done = False
        else:
            current_month = driver.find_elements(By.CLASS_NAME, "flatpickr-current-month")[0].text.strip()

        days_available = month.find_elements(By.CSS_SELECTOR, "span[data-test-id=ba-calendar-day-available]") 
        for day in days_available:
            current_month_done = True
            if len(day.text) > 0:
                try:
                    day_date_str = f"{current_month} {day.text.strip()}, {current_year}"
                    day_date_obj = datetime.datetime.strptime(day_date_str, '%B %d, %Y')
                    # Append date to complete in run based on maxium dates
                    if day_date_obj <= picked_max_date_obj and day_date_obj >= date_today_obj:
                        #If was done already some part remove days which are done
                        if is_done:
                            if day_date_obj.date() >= max_date_done:
                                days_to_complete.append(day_date_str)
                        else:
                            days_to_complete.append(day_date_str)
                except ValueError as e:
                    print(f"Error parsing date: {str(e)}")

        #### Get day which are not available
        days_not_available_elements = month.find_elements(By.CSS_SELECTOR, "span[data-test-id=ba-calendar-day]")
        days_not_available = []
        for day_not_available in days_not_available_elements:
            if len(day_not_available.text) > 0 and int(day_not_available.text) >= int(date_today.split('-')[-1]):
                days_not_available.append(f"{current_year}-{(datetime.datetime.strptime(current_month, '%B').month):02d}-{int(day_not_available.text):02d}")
        
        for day in days_not_available:
            list_of_items = process_days_not_available(activity_title=activity_title, adults_amount=adults_amount, language=language, option_date=day, url=url_id, viewer=viewer)
            df = pd.DataFrame(list_of_items)
            df = save_and_erase_dataframe(df, url_city_id, url_unique_identifier)
        ####    
        for day in days_to_complete:
            try:
                day_js = month.find_element(By.CSS_SELECTOR, f"span[aria-label*='{day}']") 
                day_js.click()
            except:
                months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
                month = months[0]
                day_js = month.find_element(By.CSS_SELECTOR, f"span[aria-label*='{day}']") 
                day_js.click()
            WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.TAG_NAME, 'details')))
            option_detials = driver.find_elements(By.TAG_NAME, 'details')
            time.sleep(2)
            list_of_items = extract_options(driver=driver, option_detials=option_detials, activity_title=activity_title, language=language, url=url_id, viewer=viewer, adults_amount=adults_amount)
            ### After extraction transform to dataframe and save it in the CSV file in case of any error in th future
            df = pd.DataFrame(list_of_items)
            # display(df)
            ## SAVE
            df = save_and_erase_dataframe(df, url_city_id, url_unique_identifier)
            check_for_modal_window_to_close(driver, calendar_picker)
            

            print(day, '----' , picked_max_day_number, day == picked_max_day_number)
            # if day_text == picked_max_day_number or day_text == month_last_day:
            #     i+=1
            #     break  
            
        i+=1
    end_time_one_link = time.time()
    logger_statistics.info(f"Time requried for {max_days_to_complete} days: {(end_time_one_link - start_time_one_link):.2f} Time reqruied for 1 day: {((end_time_one_link - start_time_one_link)/max_days_to_complete):.2f}")
    

    # quit_driver(driver)




In [13]:
# combine processed files:
def process_csv_files(folder_path, adults, language):
    # Determine today's date in the desired format    

    output_file_path = os.path.join(folder_path, f"{extraction_date_save_format}_future_price.xlsx")
    if os.path.exists(output_file_path):
        print("Output file already exists. Exiting function.")
        return
    # Ensure the archive directory exists
    archive_path = os.path.join(folder_path, "archive")
    os.makedirs(archive_path, exist_ok=True)
    
    # Initialize an empty DataFrame to hold all the data
    combined_df = pd.DataFrame()
    
    # Iterate over files in the folder
    for filename in os.listdir(folder_path):
        if filename.startswith(date_today) and filename.endswith('.csv') and f"{language}_{str(adults)}" in filename:
            file_path = os.path.join(folder_path, filename)
            
            # Read the CSV file and append its contents to the DataFrame
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            
            # Move the processed file to the archive folder
            shutil.move(file_path, os.path.join(archive_path, filename))
    
    # Save the combined DataFrame to a new CSV file
    combined_df.to_excel(output_file_path, index=False)

    print(f"All data has been combined and saved to {output_file_path}.")


In [14]:
# get_future_price(language='en', adults_amount='2', max_days_to_complete=2)
def upload_excel_to_azure_storage_account(local_file_path, storage_account_name, storage_account_key, container_name_raw, blob_name):
    try:
        # Create a connection string to the Azure Storage account
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"

        # Create a BlobServiceClient object using the connection string
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)

        # Get a reference to the container
        container_client = blob_service_client.get_container_client(container_name_raw)

        # Upload the file to Azure Blob Storage
        with open(local_file_path, "rb") as file:
            container_client.upload_blob(name=blob_name, data=file)
        create_log_done('Raw')
        print("File uploaded successfully to Azure Blob Storage (raw).")

    except Exception as e:
        print(f"An error occurred: {e}")

In [15]:
def extract_date_from_price(text):
    if 'Next available date:' in str(text):
        # Extract the date using string manipulation
        date_part = text.split('Next available date: ')[1].strip()
        
        # Parse the date string into a datetime object
        try:
            date_obj = datetime.datetime.strptime(date_part, "%A, %B %d, %Y")
            # Format the datetime object to "YYYY-MM-DD"
            return date_obj.strftime("%Y-%m-%d")
        except ValueError:
            print("Date format mismatch or error in parsing date.")
            return np.nan
    else:
        return np.nan
    
# Dynamic function to check the 'option' and set message
def dynamic_message_option(option):
    # Define patterns and their corresponding messages
    messages = {
        'Please select 1 participants or fewer for this activity.': 'set adults to 1',
        'Maximum 2 adults allowed per booking': 'adults set to 2'
        # Add more patterns and messages as needed
    }
    
    # Check each pattern and return the corresponding message if found
    for pattern, message in messages.items():
        if pattern in str(option):
            return message
    
    # Default return if no patterns matched
    return np.nan

def set_to_long_price_to_nan(text):
    text = str(text)
    if len(text) > 15 or len(text) == 0 or text == 'nan':
        return np.nan
    else:
        return text

In [16]:
def transform_upload_to_refined(local_file_path, storage_account_name, storage_account_key, container_name_refined, blob_name):
    # Define the Azure Blob Storage connection details
    connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
    # Read the Excel file into a Pandas DataFrame
    df = pd.read_excel(local_file_path)
    # city replacment if there are incorrect in url
    
    # Make changes to the df DataFrame as needed
    df['extraction_date'] = df['extraction_date'].astype('str')
    df['date'] = df['date'].astype('str')
    df['availability'] = df['total_price'].apply(extract_date_from_price)
    df['message'] = df['total_price'].apply(dynamic_message_option)
    df['total_price'] = df['total_price'].apply(set_to_long_price_to_nan)
    df['price_per_person'] = df['price_per_person'].replace('Not Available', np.nan)
    df['total_price'] = df['total_price'].str.replace(r'[$€£]', '', regex=True).str.replace(',', '').str.strip()
    # The first letter is capitalized
    df['city'] = df['city'].str.title()
    # df['total_price'] = df['total_price'].map(lambda x: x.split(x[0]) [1].strip() if not x[0].isnumeric() else x)
    # df['price_per_person'] = df['price_per_person'].map(lambda x: x.split(x[0])[1].strip() if not x[0].isnumeric() else x))

    # Save modified DataFrame to an Excel file temporarily
    output_file_path = "temp_modified_excel.xlsx"
    df.to_excel(output_file_path, index=False)
    # Create a connection to Azure Blob Storage
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        container_client = blob_service_client.get_container_client(container_name_refined)
        with open(output_file_path, "rb") as data:
            container_client.upload_blob(name=blob_name, data=data, overwrite=True)
        print("File uploaded successfully to Azure Blob Storage (refined).")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Optional: Remove the temporary file if no longer needed
        os.remove(output_file_path)

    # Function for logging (create_log_done) needs to be defined or replaced
    create_log_done('Refined')

    return 'Added to Blob'


In [17]:
def in_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [18]:
###
### TO DO: CONFIFURE THE SCRIPT BELOW TO RUN ON BETTER SCHEDULE. AS OF NOW IF THE WEEKLY UPDATE (4) WILL BE SAME DAY AS MONTHLY (1) THE WEEKLY UPDATE 
### WILL BE DONE 3 TIMES A MONTH NOT 4 WHERE IT SHOULD RUN ON BASED FREQEUNCY IF REQEUSTED WEEKLY IN A MONTH THE UDPATE SHOULD BE 
### TRIGGERED 4 TIMES IN MONTH EVEN IF THE DAY IS THE SAME AS HIGEHR REFRESH, 
### THEN IT SHOUDL RUN LATER IN MONTH TO KEEP AMOIUNT OF REFREHSES AS REQESUTED PER CLIENT
###

def should_run_today(day, month_length, frequency):
    if frequency == 1:
        # Run only on the first day of the month
        if day == 2:
            return True
    else:
# Check if the frequency is greater than current month lenght if so assing the highest value to frequency. Otherwise the interval will be 0 so it will run only on 1st day
        if frequency > month_length:
            frequency = month_length
        interval = month_length // frequency
        # Run multiple times a month
        for i in range(frequency):
            if day == 1 + i * interval:
                return True
        return False

def get_highest_order_schedule(schedules):
    today = datetime.datetime.today()
    day = today.day
    month_length = calendar.monthrange(today.year, today.month)[1]
    
    # Sort schedules by frequency in ascending order
    sorted_schedules = sorted(schedules.items(), key=lambda x: int(x[0]), reverse=False)
    for freq, value in sorted_schedules:
        frequency = int(freq)
        if should_run_today(day, month_length, frequency):
            # logger_info.info(f"Today is a run day for frequency {frequency} times a month with value {value}")
            return freq, value

    return "No schedule for today", None

In [19]:
if in_notebook():
    # Simulate the command-line arguments for Jupyter Notebook
    sys.argv = ['script_name', '--ADULTS', '4', '--LANGUAGE', 'en', '--MAX_DAYS', '5']

In [20]:
# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description="Get future price based on input parameters")
#     parser.add_argument('--ADULTS', type=str, default='2', help='Amount of adults')
#     parser.add_argument('--LANGUAGE', type=str, default='en', help='Language preference')
#     parser.add_argument('--MAX_DAYS', type=int, default=2, help='Maximum days to complete')

#     args = parser.parse_args() 

#     print(args.ADULTS, args.LANGUAGE, args.MAX_DAYS)
#     configure_dates_and_file_names(args.ADULTS, args.LANGUAGE)
#     define_logging()
#     get_future_price(adults_amount=args.ADULTS, language=args.LANGUAGE, max_days_to_complete=args.MAX_DAYS)
#     process_csv_files(output_gyg)
#     upload_excel_to_azure_storage_account(output_file_path, storage_account_name, storage_account_key, container_name_raw, blob_name)
#     transform_upload_to_refined(output_file_path, storage_account_name, storage_account_key, container_name_refined, blob_name)

In [21]:
# constant_file_path()

# with open(link_file_path) as f:
#     config = json.load(f)

# define_logging()
# # driver = initilize_driver()
# combinations = set()
# for site in config['urls']:
#     url = site['url']
#     viewer = site["viewer"]
#     for config in site['configurations']:
#         adults = config['adults']
#         language = config['language']
#         schedules = config['schedules']
        
#         schedule, max_days = get_highest_order_schedule(schedules)
#         if schedule == "No schedule for today":
#             logger_done.info(f"URL: {url} is not scheduled for today to run")
#             continue
#         configure_dates_and_file_names(adults, language)
#         #Check if current day was done and its in Archive folder
#         today_file_in_arhcive = check_if_today_done_on_schedule(url=url, schedule=schedule)
#         if today_file_in_arhcive:
#             logger_done.info(f"File in archive for URL: {url}, Adults: {adults}, Language: {language} ")
#         else:
#             print(f"Running script for URL: {url}, Adults: {adults}, Language: {language}, Max Days: {max_days}")

In [22]:
constant_file_path()

with open(link_file_path) as f:
    config = json.load(f)

define_logging()
driver = initilize_driver()
combinations = set()
for site in config['urls']:
    url = site['url']
    viewer = site["viewer"]
    for config in site['configurations']:
        adults = config['adults']
        language = config['language']
        schedules = config['schedules']
        
        schedule, max_days = get_highest_order_schedule(schedules)
        if schedule.lower() == "no schedule for today":
            logger_done.info(f"URL: {url} is not scheduled for today to run")
            break
       #Check if current day was done and its in Archive folder
        configure_dates_and_file_names(adults, language)
        today_file_in_arhcive = check_if_today_done_on_schedule(url=url, schedule=schedule)
        if today_file_in_arhcive:
            logger_done.info(f"File in archive for URL: {url}, Adults: {adults}, Language: {language} ")
            
        else:
            logger_done.info(f"Running script for URL: {url}, Adults: {adults}, Language: {language}, Max Days: {max_days}")
            get_future_price(driver=driver, adults_amount=adults ,language=language, max_days_to_complete=max_days, url=url, viewer=viewer)
            # Store the combination for later processing
        combinations.add((adults, language))

quit_driver(driver)

for adults, language in combinations:
    configure_dates_and_file_names(adults, language)
    process_csv_files(output_gyg, adults, language)
    upload_excel_to_azure_storage_account(output_file_path, storage_account_name, storage_account_key, container_name_raw, blob_name)
    transform_upload_to_refined(output_file_path, storage_account_name, storage_account_key, container_name_refined, blob_name)

2024-09-24 14:24:26,526 - Info_logger - INFO - Initializing the Chrome driver and logging into the website
2024-09-24 14:24:27,952 - Info_logger - INFO - File_pattern: G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price/Archive/2024-09-24_*-*-*_en_2-lisbon-l42-GYG.csv
2024-09-24 14:24:27,958 - Info_logger - INFO - matching_files: ['G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price/Archive\\2024-09-24_13-00-00_en_2-lisbon-l42-GYG.csv']
2024-09-24 14:24:27,975 - Done_logger - INFO - Running script for URL: https://www.getyourguide.com/lisbon-l42/lisbon-pena-palace-sintra-cabo-da-roca-cascais-daytrip-t387617/, Adults: 2, Language: en, Max Days: 5
2024-09-24 14:24:27,978 - Info_logger - INFO - Adults amount: 2
2024-09-24 14:24:27,979 - Info_logger - INFO - Language: en
2024-09-24 14:24:27,980 - Info_logger - INFO - Max days to complete: 5
2024-09-24 14:24:28,969 - Info_logger - I

Element with data-test-id 'activity-filters-primary-language-picker' exists.
Current date: 26


2024-09-24 14:24:33,474 - Done_logger - INFO - Sucessfuly extracted | English Tour without Pena Palace Entry Ticket | Not listed | €132.00 | 66.0 |
2024-09-24 14:24:33,508 - Done_logger - INFO - Sucessfuly extracted | Tour in English and Pena Palace Gardens Entry Ticket | Not listed | €148.80 | 74.4 |
2024-09-24 14:24:33,544 - Done_logger - INFO - Sucessfuly extracted | English Tour with Pena Palace Entry Ticket | Not listed | €161.98 | 80.99 |
2024-09-24 14:24:33,578 - Done_logger - INFO - Sucessfuly extracted | Private English Tour with Pena Palace Ticket | Not listed | Next available date: Friday, September 27, 2024 | Not Available |
2024-09-24 14:24:33,601 - Done_logger - INFO - Sucessfully upserted 4 rows to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price/2024-09-24_14-00-00_en_2-lisbon-l42-GYG.csv
2024-09-24 14:24:34,025 - Done_logger - INFO - Sucessfully upserted 1 rows to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SM

September 27, 2024 ---- 29 False


2024-09-24 14:24:40,503 - Done_logger - INFO - Sucessfuly extracted | English Tour without Pena Palace Entry Ticket | Not listed | €132.00 | 66.0 |
2024-09-24 14:24:40,543 - Done_logger - INFO - Sucessfuly extracted | Tour in English and Pena Palace Gardens Entry Ticket | Not listed | €148.80 | 74.4 |
2024-09-24 14:24:40,582 - Done_logger - INFO - Sucessfuly extracted | English Tour with Pena Palace Entry Ticket | Not listed | €161.98 | 80.99 |
2024-09-24 14:24:40,618 - Done_logger - INFO - Sucessfuly extracted | Private English Tour with Pena Palace Ticket | Not listed | Next available date: Tuesday, October 1, 2024 | Not Available |
2024-09-24 14:24:40,623 - Done_logger - INFO - Sucessfully upserted 4 rows to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price/2024-09-24_14-00-00_en_2-lisbon-l42-GYG.csv
2024-09-24 14:24:40,664 - Satistics_logger - INFO - Time requried for 5 days: 12.68 Time reqruied for 1 day: 2.54
2024-09-24 14:

...

Current currency is  EUR
Element with data-test-id 'activity-filters-primary-language-picker' exists.
Current date: 26


2024-09-24 14:24:54,047 - Done_logger - INFO - Sucessfuly extracted | English Tour without Pena Palace Entry Ticket | Not listed | €264.00 | 66.0 |
2024-09-24 14:24:54,082 - Done_logger - INFO - Sucessfuly extracted | Tour in English and Pena Palace Gardens Entry Ticket | Not listed | €297.60 | 74.4 |
2024-09-24 14:24:54,118 - Done_logger - INFO - Sucessfuly extracted | English Tour with Pena Palace Entry Ticket | Not listed | €323.96 | 80.99 |
2024-09-24 14:24:54,148 - Done_logger - INFO - Sucessfuly extracted | Private English Tour with Pena Palace Ticket | Not listed | Next available date: Friday, September 27, 2024 | Not Available |
2024-09-24 14:24:54,156 - Done_logger - INFO - Sucessfully upserted 4 rows to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price/2024-09-24_14-00-00_en_4-lisbon-l42-GYG.csv
2024-09-24 14:24:54,589 - Done_logger - INFO - Sucessfully upserted 1 rows to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SM

September 27, 2024 ---- 29 False


2024-09-24 14:25:01,036 - Done_logger - INFO - Sucessfuly extracted | English Tour without Pena Palace Entry Ticket | Not listed | €264.00 | 66.0 |
2024-09-24 14:25:01,074 - Done_logger - INFO - Sucessfuly extracted | Tour in English and Pena Palace Gardens Entry Ticket | Not listed | €297.60 | 74.4 |
2024-09-24 14:25:01,114 - Done_logger - INFO - Sucessfuly extracted | English Tour with Pena Palace Entry Ticket | Not listed | €323.96 | 80.99 |
2024-09-24 14:25:01,145 - Done_logger - INFO - Sucessfuly extracted | Private English Tour with Pena Palace Ticket | Not listed | Next available date: Tuesday, October 1, 2024 | Not Available |
2024-09-24 14:25:01,150 - Done_logger - INFO - Sucessfully upserted 4 rows to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price/2024-09-24_14-00-00_en_4-lisbon-l42-GYG.csv
2024-09-24 14:25:01,187 - Satistics_logger - INFO - Time requried for 5 days: 10.02 Time reqruied for 1 day: 2.00


September 28, 2024 ---- 29 False
All data has been combined and saved to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price\2024-09-24_14-00-00_en_3_future_price.xlsx.
File uploaded successfully to Azure Blob Storage (raw).
File uploaded successfully to Azure Blob Storage (refined).
All data has been combined and saved to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price\2024-09-24_14-00-00_en_4_future_price.xlsx.
File uploaded successfully to Azure Blob Storage (raw).
File uploaded successfully to Azure Blob Storage (refined).
All data has been combined and saved to G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price\2024-09-24_14-00-00_en_2_future_price.xlsx.
File uploaded successfully to Azure Blob Storage (raw).
File uploaded successfully to Azure Blob Storage (refined).


{(2, 'en'), (3, 'en'), (4, 'en')}